In [2]:
import sys
import os

sys.path.append(os.path.abspath(".."))

DATABASE_DIR_MANIFESTOS = "../../data/manifestos/chroma/manifestoberta"
DATABASE_DIR_DEBATES = "../../data/manifestos/chroma/manifestoberta"
TEST_DATA_DIR = "../../data/questions"

In [3]:
import numpy as np
import pandas as pd

from database.vector_database import VectorDatabase
from models.embedding import ManifestoBertaEmbeddings

# Create simple chain

In [4]:
from models.generation import generate_chain_with_balanced_retrieval

embedding_model = ManifestoBertaEmbeddings()

db_manifestos = VectorDatabase(
    data_path="../data/manifestos",
    embedding_model=embedding_model,
    database_directory=DATABASE_DIR_MANIFESTOS,
    source_type="manifestos",
)

chain = generate_chain_with_balanced_retrieval(
    [db_manifestos]
)  # add more databases here

/Users/christianliedl/anaconda3/envs/bundestag/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of XLMRobertaModel were not initialized from the model checkpoint at manifesto-project/manifestoberta-xlm-roberta-56policy-topics-sentence-2023-1-1 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


reloaded database


In [5]:
question = "Was sind die Position der Partein zur Rentenpolitik?"
response = chain.invoke(question)
response

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{'cdu_summary': 'Die CDU/CSU setzt sich für eine starke Sozialstaatlichkeit ein und betont die Bedeutung sozialer Sicherungssysteme. Sie lehnt eine Einflussnahme der EU auf den Betrieb und die Finanzierung von Krankenhäusern in Deutschland ab und spricht sich gegen eine weitere Privatisierung im Gesundheitswesen aus. Zudem betont sie die Notwendigkeit, den Zugang zu Arzneimitteln für seltene Erkrankungen sicherzustellen.',
 'spd_summary': 'Die SPD setzt sich für die Überwindung von Wohnungslosigkeit bis 2030 ein und unterstützt die Europäische Plattform zur Bekämpfung von Wohnungslosigkeit. Sie strebt einen sicheren Zugang zu Gesundheitsversorgung für alle Bürger in Europa an und fordert eine weitere Stärkung der Pflege durch staatliche Investitionen und verbindliche europäische Regelungen.',
 'gruene_summary': 'Bündnis 90/Die Grünen legen den Fokus auf Prävention und Gesundheitsförderung in ihrer Gesundheitspolitik. Sie setzen sich für die Stärkung der Kinderrechte ein und betonen die

# Load test data

In [6]:
import pandas as pd

df_test_simple = pd.read_csv(os.path.join(
    TEST_DATA_DIR, "simple_questions.csv"))

df_test_complex = pd.read_csv(os.path.join(
    TEST_DATA_DIR, "complex_questions.csv"))

# Create Eval chain

In [7]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    context_recall,
)
from ragas.langchain import RagasEvaluatorChain

# create eval chains
eval_chains = {m.name: RagasEvaluatorChain(
    metric=m) for m in [context_relevancy]}

# Run eval chain for test data

In [57]:
metrics = {}
for party in sources:
    for name, _ in eval_chains.items():
        metrics.update({f"{name}_score_{party}": []})

sources = ["gruene", "spd", "cdu", "afd", "fdp", "linke"]

for question in df_test_simple["EINFACHE FRAGEN"]:
    print("_____________________________________")
    print(question)
    # Rename keys to be compatible with eval_chain
    output = {}
    output["query"] = question

    for party in sources:
        output["source_documents"] = db_manifestos.database.similarity_search(
            query=question, k=5, filter={"party": party}
        )
        output["result"] = ""

        # Evaluate one metric at a time
        for name, eval_chain in eval_chains.items():
            score_name = f"{name}_score"
            scores = eval_chain(output)[score_name]
            print(f"{score_name}_{party}: {scores}")
            metrics[f"{score_name}_{party}"].append(scores)

_____________________________________
Was denken die Parteien darüber, mehr Geld für die Armee auszugeben, um Europa sicher zu halten?
context_relevancy_score: 0.0
context_relevancy_score: 0.05128205128205128
context_relevancy_score: 0.11458333333333333
context_relevancy_score: 0.03278688524590164
context_relevancy_score: 0.0
context_relevancy_score: 0.0
_____________________________________
Was sagen die Parteien, wie wir Transgender-Menschen helfen und dafür sorgen können, dass alle fair behandelt werden?
context_relevancy_score: 0.11494252873563218
context_relevancy_score: 0.10256410256410256
context_relevancy_score: 0.0
context_relevancy_score: 0.0
context_relevancy_score: 0.12903225806451613
context_relevancy_score: 0.2222222222222222
_____________________________________
Sollen wir in Europa mehr für den Schutz unserer Umwelt tun?
context_relevancy_score: 0.06796116504854369


KeyboardInterrupt: 

In [12]:
from openai import OpenAI

client = OpenAI()

for question in df_test_simple["EINFACHE FRAGEN"].iloc[:1]:

    sources = ["gruene", "spd", "cdu", "afd", "fdp", "linke"]

    print("_____________________________________")
    print(question)
    # Rename keys to be compatible with eval_chain
    output = {}
    output["query"] = question

    for party in sources:
        output["source_documents"] = db_manifestos.database.similarity_search(
            query=question, k=5, filter={"party": party}
        )

        instruction = """Du hilfst mir bei der Evaluation eines RAG systems. Bewerte, ob die folgenden Dokumente relevant sind für die Frage. Antworte mit einer List, in der für jedes Dokument entweder 0 (nicht relevant) oder 1 (relevant) ausgegeben wird.
        Beispiel: [0, 1, 1, 1, 0]"""

        context = ""
        for i, doc in enumerate(output["source_documents"]):
            context += f"""Dokument {i+1}: {doc.page_content}\n\n\n
            """
        prompt = f"{instruction}\n\n\nHier sind die Dokumente:\n{context}\n\n\nHier ist die Frage:{question}"

        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[
                {"role": "system", "content": prompt},
            ],
        )

        print(completion.choices[0].message)

_____________________________________
Was denken die Parteien darüber, mehr Geld für die Armee auszugeben, um Europa sicher zu halten?
ChatCompletionMessage(content='1: 1\n2: 1\n3: 0\n4: 0\n5: 1', role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='1, 1, 0, 1, 0', role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='1, 1, 1, 0, 1', role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='1, 0, 1, 1, 0', role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='1, 2, 3, 4, 5', role='assistant', function_call=None, tool_calls=None)
ChatCompletionMessage(content='1, 1, 1, 1, 0', role='assistant', function_call=None, tool_calls=None)


ChatCompletionMessage(content='1, 1, 1, 1, 1', role='assistant', function_call=None, tool_calls=None)


In [37]:
df_metrics = pd.DataFrame(metrics)
df_metrics.loc[:, "question"] = df_test_simple["EINFACHE FRAGEN"]
df_metrics.to_csv("metrics.csv")

In [41]:
df_metrics = pd.read_csv("metrics.csv")

{key + "_mean": np.mean(df_metrics[key]) for key in metrics.keys()}

{'faithfulness_mean': 0.6958333333333333,
 'answer_relevancy_mean': 0.8132587018852587,
 'context_relevancy_mean': 0.08362864874686092}